# **Ising Model in 2D**

<i class="fa fa-home fa-2x"></i><a href="./index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/ising_model.ipynb

The Cython computing kernel is modified from the code, which presents in the 
blog [Pythonic Perambulations](https://jakevdp.github.io/blog/2017/12/11/live-coding-cython-ising-model).

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

<p style="text-align: justify;font-size:15px">  
    This notebook demonstrates the Ising model for a two dimensional system. Ising model is a
    mathematical model of ferromagnetism in statistical mechanics, which is named after the
    physicist Ernst Ising.
</p>

<p style="text-align: justify;font-size:15px">
    In a magnetic material system, each atom can be in spin up (+) and down (-) states. 
    The energy of a configuration can be described by a Hamiltonian function, which is 
    shown below:
</p>

$\large H = - \dfrac{1}{2} \sum_{i,j} J\sigma_i \sigma_j $ (1)

<p style="text-align: justify;font-size:15px">
    where $J_{ij}$ the strength of exchange interaction. Each atom only interects with its
    nearest-neighbor. Each $\sigma_i$ can be up (+1) and down (-1). Here, we are using 
    Monte Carlo simulation to study the spin configurations of the material systems.
</p>

## **Monte Carlo agorithm**

<p style="text-align: justify;font-size:15px">
<ol>
    <li>Start with some spin configuration.</li>
    <li>Randomly pick a site and consider flipping the spin over on that site.</li>
    <li>Compute energy for that preturbation.</li>
    <li>If $\delta E<0$ accept perturbation, if $\delta E>0$ accept perturbation with
        probability $exp\left[\frac{-\delta E}{kT}\right]$. </li>
    <li>Go back to step 2. </li>
</ol>    
</p>

## **Instruction for using this notebook**

<p style="text-align: justify;font-size:15px">
    The figure on the left shows the spin configurations. Yellow pixel represents one spin 
    up and deep blue represents one spin down. You can set the exchange interaction parameter 
    J by tunning the slider. The "N" slider defines the number of the spins in one dimension. 
    The figure on the right shows the average spin changing with the Monte Carlo simulations. 
    By clicking "Run simulation", it will generate a random initial configuration and simulation
    one hundred steps. You can view the evolution of the simulations by the "Frame" slider.
</p>

<p style="text-align: justify;font-size:15px">
    Select J value as negative, positive and zero number and run the simulations. 
    What is the different of these spin configurations respectively ? And why ?
</p>

In [ ]:
%reload_ext Cython
%matplotlib widget

In [ ]:
import numpy as np
from ipywidgets import interact, FloatSlider, Button, Output, IntSlider, VBox
from ipywidgets import HBox, Checkbox, IntProgress, HTML
import matplotlib.pyplot as plt
from time import sleep
import matplotlib.gridspec as gridspec
from scipy.ndimage import convolve
from matplotlib.animation import FuncAnimation
import base64

In [ ]:
def random_spin_field(N, M):
    """Randomize the initial spin configuration."""
    return np.random.choice([-1, 1], size=(N, M))

def all_up_spin_field(N, M):
    """Set all spin up."""
    return np.ones((N, M), dtype=int)

run_button = Button(description='Run simulation')
run_button.style.button_color = 'lightgreen'
play_button = Button(description='Play', disabled=True)

random_checkbox = Checkbox(value=False, description="Start random")

jvalue_slider = FloatSlider(value = 1.0, min = -2.0, max = 2.0, description = 'Exchange interaction J:',
                    style={'description_width': 'initial'})
num_slider = IntSlider(value=100, min=50, max=300, step=10, description="Size")
temp_slider = FloatSlider(value=2, min=0.5, max=4, step=0.1, description="Temperature") # Units of J/k
step_slider = IntSlider(value=100, min=100, max=1000, step=50, description="Num steps")
frame_slider = IntSlider(value=0, min=0, max=step_slider.value, description="Frame", layout={'width':'800px'}, disabled=True)

In [ ]:
%%cython

cimport cython

import numpy as np
cimport numpy as np

from libc.math cimport exp
from libc.stdlib cimport rand
cdef extern from "limits.h":
    int RAND_MAX

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step(np.int64_t[:, :] field, float beta, float J):
    """Update the Ising step, each step actually contains N*M steps.
       
    Args:
        field: matrix representation for the spin configuration.
        beta: 1/kbT
        J: the strength of exchange interaction.
        
    Returns:
        New spin configuration and the total energy change.
    """
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i
    cdef np.ndarray x, y
    cdef float dE = 0.0
    
    x = np.random.randint(N, size=(N*M))
    y = np.random.randint(M, size=(N*M))
    
    for i in range(N*M):
        dE += _cy_ising_update(field, x[i], y[i], beta, J)
        
    return np.array(field), dE

@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update(np.int64_t[:, :] field, int n, int m, float beta, float J):
    """Monte Carlo simulation using the Metropolis algorithm.
    
    Args:
        field: matrix representation for the spin configuration.
        n: choosed row index.
        m: choosed column index.
        beta: 1/kbT
        J: the strength of exchange interaction.
        
    Returns:
        The total energy change.
    """
    cdef int total
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]

    total = field[(n+1)%N, m] + field[n, (m+1)%M] + field[(n-1)%N, m] + field[n, (m-1)%M]
    cdef float dE = 2.0 * J * field[n, m] * total
    if dE <= 0:
        field[n, m] *= -1
        return dE
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n, m] *= -1
        return dE
    else:
        return 0

In [ ]:
pause = True;

def on_frame_change(b):
    """Update the plot for playing the animation."""
    global fig, v1
    fig.set_data(images[frame_slider.value])
    v1.set_data([frame_slider.value, frame_slider.value],[-1.1, 1.1])
    v2.set_data([frame_slider.value, frame_slider.value],[-5000, 5000])

    
def compute_total_energy(M, J):
    """Compute the total energy of the given spin configuration."""
    a = np.ones(np.shape(M));
    c = convolve(M, a, mode='constant')
    c = (c-M)*M*J
    return c.sum()


def update(frame):
    """Update function for the animation."""
    global pause
    
    if pause:
        ani.event_source.stop()
    else:
        frame_slider.value = frame
        return (fig)
    
def play_animation(event):
    """OnClick function the 'Play' button."""
    global pause
    
    pause ^= True
    if play_button.description == "Pause":
        play_button.description = "Play"
        ani.event_source.stop()
    else:
        play_button.description = "Pause"
        ani.event_source.start()
    

frame_slider.observe(on_frame_change, names='value')

images = [all_up_spin_field(num_slider.value, num_slider.value)]

img = plt.figure(tight_layout=True, figsize=(8,5))
img.canvas.header_visible = False
gs = gridspec.GridSpec(4, 2)

ax1 = img.add_subplot(gs[:, 0])
ax2 = img.add_subplot(gs[0:2, 1])
ax3 = img.add_subplot(gs[2:4, 1])

fig = ax1.imshow(images[0], vmin=-1, vmax=1)

line1, = ax2.plot([0], [0], 'r-')
line2, = ax3.plot([0], [0], 'r-')
v1 = ax2.axvline(x=0, c='black')
v2 = ax3.axvline(x=0, c='black')

ax2.set_xlim([0, step_slider.value])
ax2.set_ylim([-1.1, 1.1])
ax2.set_title('Average spin', fontsize=12)

ax3.set_xlim([0, step_slider.value])
ax3.set_ylim([-1, 1])
ax3.set_title('Total energy', fontsize=12)

ani = FuncAnimation(img, update, interval= 20, frames=np.arange(0, step_slider.value+1), blit=True)

def run_simulation(b):
    """OnClick function the 'Run simulation' button."""
    play_button.disabled = True
    run_button.disabled = True
    frame_slider.disabled = True
    run_button.style.button_color = 'red'
    global images, fig
    if random_checkbox.value:
        images = [random_spin_field(num_slider.value, num_slider.value)]
    else:
        images = [all_up_spin_field(num_slider.value, num_slider.value)]
        
    
    x = np.arange(step_slider.value + 1);
    y1 = [];
    y2 = [0];
    
    for i in range(step_slider.value):
        imag, dE = cy_ising_step(images[-1].copy(), beta=1.0/temp_slider.value, J=jvalue_slider.value)
        images.append(imag)
        y2.append(dE+y2[-1])
        
    frame_slider.max = step_slider.value
    ax2.set_xlim([0, step_slider.value])
    
    fig.set_data(images[0])
    
    for i in images:
        y1.append(i.sum()*1.0/(num_slider.value * num_slider.value))
    
    y1 = np.array(y1)
    y2 = np.array(y2)
    
    line1.set_data(x, y1)
    line2.set_data(x, y2)
    
    ax3.set_ylim([y2.min(), y2.max()])
    ax3.set_xlim([0, step_slider.value])
    
    frame_slider.value = 0
    
    ani.frames = np.arange(0, step_slider.value+1)
    ani._iter_gen = lambda: iter(ani.frames)
    ani.save_count = len(ani.frames)
    ani.frame_seq = ani.new_frame_seq()

    frame_slider.disabled = False
    play_button.disabled = False
    run_button.disabled = False
    run_button.style.button_color = 'lightgreen'
    return y1

run_button.on_click(run_simulation);
play_button.on_click(play_animation);

display(frame_slider,
    HBox([num_slider, step_slider]),
    HBox([jvalue_slider, temp_slider]),
    HBox([random_checkbox, run_button, play_button]))

In [ ]:
temps = []
average_spins = []
error_spins = []

progressbar = IntProgress(value=0, min=0, max=49, description='Computing:', bar_style='warning')

def run_all(min_temp=1.5, max_temp=4, num_reps = 20, num_temps = 50):
    """Scan the temperature and compute the average magnetization per spin."""
    global Temp
    
    temps = []
    average_spins = []
    error_spins = []
    i = 0

    for temperature in np.linspace(min_temp, max_temp, num_temps):
        progressbar.value = i
        i += 1
        temp_slider.value = temperature
        accum = []
        for iter_id in range(num_reps):
            # Store absolute value: when starting with random, I might get either up or down "randomly"
            accum.append(abs(run_simulation(None)[-1]))
        accum = np.array(accum)
        y1 = accum.mean()
        y1err = accum.std()
        #print(temperature, y1, y1err)
        temps.append(temperature)
        average_spins.append(y1)
        error_spins.append(y1err)
    return temps, average_spins, error_spins

def plot_transition(temps, average_spins, error_spins):
    """Plot the average magnetization per spin as a function of the temperature."""
    figt, axt = plt.subplots()
    axt.errorbar(temps, average_spins, error_spins, fmt='-o')
    axt.set_xlabel("$k_B T / |J|$")
    axt.set_ylabel(r"$\langle \sigma \rangle$")
    figt.savefig('Ising.pdf')


def run_all_callback(b):
    """OnClick function for the 'Compute Tc' button."""
    global temps, average_spins, error_spins

    temps, average_spins, error_spins = run_all()
    plot_transition(temps, average_spins, error_spins)
    download_button.value = _download_button_format.format(data=_prepare_payload(), filename='Ising.pdf', encoding='base64', disabled='', button_style='default')

runall_button = Button(description=r'Compute Tc')
runall_button.on_click(run_all_callback)

def _prepare_payload(file_format=None):
    """Prepare binary information."""
    with open('Ising.pdf', 'rb') as raw:
        return base64.b64encode(raw.read()).decode()

_download_button_format = """
<input type="button" class="p-Widget jupyter-widgets jupyter-button widget-button mod-{button_style}" value="Download Image" title="Download structure" style="width:auto;" {disabled}
onclick="var link = document.createElement('a');
link.href = 'data:charset={encoding};base64,{data}';
link.download = '{filename}';
document.body.appendChild(link);
link.click();
document.body.removeChild(link);" />
"""

download_button = HTML(_download_button_format.format(data='', filename='', encoding='base64', disabled='disabled', button_style='default'))
display(HBox([progressbar, runall_button, download_button]))